<a href="https://colab.research.google.com/github/RCS007/Oasis-_Web-Devlpoment_Level1_Task1/blob/main/Movie_Recommendation_System_By_Using_KNN_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Basic Libraries 

In [4]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process

Loading Datasets

In [15]:
movie = pd.read_csv('/content/movie.csv')
rating = pd.read_csv('/content/rating.csv')
tag = pd.read_csv('/content/tag.csv')

In [16]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [17]:
movie['year'] = movie.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movie['year'] = movie.year.str.extract('(\d\d\d\d)',expand=False)

In [18]:
#Removing the years from the 'title' column
movie['title'] = movie.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movie['title'] = movie['title'].str.strip()
movie.head()

<ipython-input-18-cfd3e354d150>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  movie['title'] = movie.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [19]:
rating.groupby('movieId').rating.mean()
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


Joining Two Datasets

In [20]:
df=movie.join(rating,lsuffix='N', rsuffix='K')
df

,movieIdN,title,genres,year,userId,movieIdK,rating,timestamp
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,1,2,3.5,2005-04-02 23:53:47
1,2,Jumanji,Adventure|Children|Fantasy,1995,1,29,3.5,2005-04-02 23:31:16
2,3,Grumpier Old Men,Comedy|Romance,1995,1,32,3.5,2005-04-02 23:33:39
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995,1,47,3.5,2005-04-02 23:32:07
4,5,Father of the Bride Part II,Comedy,1995,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...,...,...,...,...
27273,131254,Kein Bund für's Leben,Comedy,2007,211,527,5.0,2006-10-08 18:35:17
27274,131256,"Feuer, Eis & Dosenbier",Comedy,2002,211,589,3.5,2006-10-08 18:37:28
27275,131258,The Pirates,Adventure,2014,211,593,4.0,2006-10-08 18:38:01
27276,131260,Rentun Ruusu,(no genres listed),2001,211,594,4.5,2006-10-08 18:31:34


Removing Unnecessary Columns

In [21]:
df=df.drop(['movieIdK','genres','year','timestamp'],axis=1)

Creating Matrix 

In [22]:
# There will be lot of nan value in our new dataframe as it is highly unlikely that that ours users have seen almost all movies.
movie_users=df.pivot(index='movieIdN', columns='userId',values='rating').fillna(0)
matrix_movies_users=csr_matrix(movie_users.values)
print(matrix_movies_users)

  (0, 0)	3.5
  (1, 0)	3.5
  (2, 0)	3.5
  (3, 0)	3.5
  (4, 0)	3.5
  (5, 0)	3.5
  (6, 0)	4.0
  (7, 0)	4.0
  (8, 0)	4.0
  (9, 0)	4.0
  (10, 0)	4.0
  (11, 0)	4.0
  (12, 0)	4.0
  (13, 0)	3.5
  (14, 0)	3.5
  (15, 0)	4.0
  (16, 0)	3.5
  (17, 0)	3.5
  (18, 0)	3.0
  (19, 0)	3.5
  (20, 0)	3.5
  (21, 0)	3.5
  (22, 0)	4.0
  (23, 0)	4.0
  (24, 0)	3.5
  :	:
  (27253, 209)	3.0
  (27254, 209)	4.0
  (27255, 209)	5.0
  (27256, 209)	4.0
  (27257, 209)	4.0
  (27258, 209)	4.0
  (27259, 209)	2.0
  (27260, 210)	3.0
  (27261, 210)	3.0
  (27262, 210)	3.0
  (27263, 210)	4.0
  (27264, 210)	4.0
  (27265, 210)	5.0
  (27266, 210)	3.0
  (27267, 210)	4.0
  (27268, 210)	5.0
  (27269, 210)	4.0
  (27270, 210)	4.0
  (27271, 210)	5.0
  (27272, 210)	3.0
  (27273, 210)	5.0
  (27274, 210)	3.5
  (27275, 210)	4.0
  (27276, 210)	4.5
  (27277, 210)	4.0


Using Cosine Matrix (KNN)

In [23]:
knn= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10)

Fitting The Model

In [24]:
knn.fit(matrix_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

In [25]:
NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

Creating Recommendation Function

In [29]:
def recommender(movie_name, data,model, n_recommendations ):
    model.fit(data)
    idx=process.extractOne(movie_name, df['title'])[2]
    print('Movie Selected:-',df['title'][idx], 'Index: ',idx)
    print('Searching for recommendations.....')
    distances, indices=model.kneighbors(data[idx], n_neighbors=n_recommendations)
    for i in indices:
        print(df['title'][i].where(i!=idx))
    
recommender('Toy Story', matrix_movies_users, knn,10)

Movie Selected:- Toy Story Index:  0
Searching for recommendations.....
117                              Steal Big, Steal Little
115                       Young Poisoner's Handbook, The
119                             Boys of St. Vincent, The
118                                         Race the Sun
121               Chungking Express (Chung Hing sam lam)
116                                         If Lucy Fell
114                                Anne Frank Remembered
122              Star Maker, The (Uomo delle stelle, L')
113    Happiness Is in the Field (Bonheur est dans le...
120                                            Boomerang
Name: title, dtype: object
